In [1]:
import sys 
import os
import re

import time
from typing import Union, Callable
from functools import wraps

import base64
import requests
from requests.auth import HTTPBasicAuth

In [2]:
project_path = os.path.abspath('..')
if project_path not in sys.path:
    sys.path.append(project_path)
sys.path

['C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner\\src',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\python311.zip',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\DLLs',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3',
 '',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib',
 'C:\\Users\\Filipp\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin',
 'C:\\Users\\Filipp\\PycharmProjects\\Invoice_scanner']

In [8]:
from config.config import config
from src.logger import logger
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
config['user_1C'] = os.getenv('user_1C')
config['password_1C'] = os.getenv('password_1C')

auth = HTTPBasicAuth(config['user_1C'], config['password_1C'])

In [10]:
def cache_http_requests(func):
    """ Декоратор для кэширования запросов на основе URL """

    cache = {}
    max_cache_size = 40

    @wraps(func)
    def wrapper(function, *args, **kwargs):
        # Формируем ключ кэша из функции + "_" + аргументы
        function_args = r'_'.join(args)
        url_cache_key = function + r'_' + function_args

        # Проверяем, есть ли результат в кэше для данного URL
        if url_cache_key in cache:
            logger.print("Получение результата из кэша...")
            return cache[url_cache_key]

        # Выполняем запрос и сохраняем результат в кэше
        result = func(function, *args, **kwargs)
        cache[url_cache_key] = result

        if len(cache) > max_cache_size:
            cache.pop(next(iter(cache)))

        return result

    return wrapper


@cache_http_requests
def cup_http_request(function, *args, kappa=False, encode_off=False) -> Union[list, dict, None]:
    user_1C = config['user_1C']
    password_1C = config['password_1C']

    # Определение серверов
    if kappa:
        primary_base = r'http://kappa5.group.ru:81/ca/hs/interaction/'
        secondary_base = r'http://10.10.0.10:81/ca/hs/interaction/'
    else:
        primary_base = r'http://10.10.0.10:81/ca/hs/interaction/'
        secondary_base = r'http://kappa5.group.ru:81/ca/hs/interaction/'

    if encode_off:
        encode_func: Callable = lambda x: x
    else:
        encode_func: Callable = lambda x: base64.urlsafe_b64encode(x.encode()).decode()

    function_args = r'/'.join(map(encode_func, args))

    try:
        # Формируем URL для первого сервера
        primary_url = primary_base + function + r'/' + function_args
        logger.print(f"Попытка запроса: {primary_url}")

        # Попытка отправить запрос на первый сервер
        response = requests.get(primary_url, auth=HTTPBasicAuth(user_1C, password_1C))

        # Если первый запрос успешен, возвращаем результат
        if response.status_code == 200:
            return response.json()
        else:
            logger.print(f"Ошибка при запросе к первому серверу: {response.status_code} - {response.reason}")
    except Exception as error:
        logger.print(error)

    try:
        # Формируем URL для второго сервера
        secondary_url = secondary_base + function + r'/' + function_args
        logger.print(f"Попытка запроса ко второму серверу: {secondary_url}")

        # Попытка отправить запрос на второй сервер
        response = requests.get(secondary_url, auth=HTTPBasicAuth(user_1C, password_1C))

        # Возвращаем результат, если успешен
        if response.status_code == 200:
            return response.json()
        else:
            logger.print(f"Ошибка при запросе ко второму серверу: {response.status_code} - {response.reason}")
            return None
    except Exception as error:
        logger.print(error)
        return None


def add_partner(response: Union[list, dict, None]):
    if not isinstance(response, list):
        return response

    regex = r'(.*) (от) (.*)'
    matches = [re.fullmatch(regex, deal, re.IGNORECASE) for deal in response]
    if all(matches):
        deals = [match.group(1) for match in matches]
        partners = [cup_http_request(r'ValueByTransactionNumber', deal, 'Контрагент') for deal in deals]
        partners = [p[0] if p else '' for p in partners]

        new_response = []
        for m, p in zip(matches, partners):
            deal_and_partner = m.group() + f" | {p}"
            new_response.append(deal_and_partner)
        return new_response

    else:
        return response


def cup_http_request_partner(function, *args, kappa=False, encode_off=False) -> Union[list, dict, None]:
    response = cup_http_request(function, *args, kappa=kappa, encode_off=encode_off)
    return add_partner(response)

In [14]:
res = cup_http_request("InvoicesItemsByDates", '01-10-2024', '10-10-2024', encode_off=True)

Получение результата из кэша...


In [16]:
res[0:3]

[{'Услуга': 'Сверхнормативное использование автотранспорта',
  'УслугаКод': 'ТК-003329',
  'Упоминаний': 2,
  'Счетов': 1},
 {'Услуга': 'Погрузка ручная ',
  'УслугаКод': 'ТК-003336',
  'Упоминаний': 1,
  'Счетов': 1},
 {'Услуга': 'Взвешивание',
  'УслугаКод': 'ТК-006728',
  'Упоминаний': 13,
  'Счетов': 9}]